In [1]:
import rospy
import intera_interface
import copy

ModuleNotFoundError: No module named 'rospy'

In [2]:
from geometry_msgs.msg import (
    PoseStamped,
    Pose,
    Point,
    Quaternion,
)
from std_msgs.msg import Header
from sensor_msgs.msg import JointState

from intera_core_msgs.srv import (
    SolvePositionIK,
    SolvePositionIKRequest,
)

In [3]:
rospy.init_node("sawyer_demo_node")

In [4]:
limb = intera_interface.Limb('right')

In [5]:
robot_state = intera_interface.RobotEnable()

In [6]:
robot_state.enable()

[INFO] [1543447425.474195, 177.074000]: Robot Enabled


In [7]:
# Joint Position Example

In [8]:
current_position = limb.joint_angles()
print current_position

{'right_j6': -0.030876760925905522, 'right_j5': -0.10264515007911701, 'right_j4': -0.01614688408149423, 'right_j3': 0.41061666992453993, 'right_j2': 0.23067483041642323, 'right_j1': 1.6174384398878479, 'right_j0': -0.1909147481041611}


In [9]:
new_position = copy.copy(current_position)
new_position['right_j0'] -=1

In [10]:
limb.move_to_joint_positions(new_position)

In [11]:
for key in new_position:
    new_position[key] = 0.0

In [12]:
limb.move_to_joint_positions(new_position)

In [ ]:
# Gripper Example

In [13]:
gripper = intera_interface.Gripper('right_gripper')

In [14]:
gripper.get_position()

0.0

In [15]:
gripper.set_position(100)


In [16]:
gripper.set_position(0)

In [51]:
# Inverse Kinematics Example

In [17]:
ik_service_name = "ExternalTools/right/PositionKinematicsNode/IKService"

In [18]:
iksvc = rospy.ServiceProxy(ik_service_name, SolvePositionIK)

In [19]:
ikreq = SolvePositionIKRequest()

In [20]:
hdr = Header(stamp=rospy.Time.now(), frame_id='base')

In [21]:
desired_pose = PoseStamped(
            header=hdr,
            pose=Pose(
                position=Point(
                    x=0.450628752997,
                    y=0.161615832271,
                    z=0.217447307078,
                ),
                orientation=Quaternion(
                    x=0.704020578925,
                    y=0.710172716916,
                    z=0.00244101361829,
                    w=0.00194372088834,
                )
            )
        )

In [22]:
ikreq.pose_stamp.append(desired_pose)

In [23]:
ikreq.tip_names.append('right_gripper_tip')

In [24]:
rospy.wait_for_service(ik_service_name,5)

In [25]:
resp = iksvc(ikreq)

In [26]:
print resp

joints: 
  - 
    header: 
      seq: 0
      stamp: 
        secs: 1543447835
        nsecs: 523935218
      frame_id: ''
    name: [right_j0, right_j1, right_j2, right_j3, right_j4, right_j5, right_j6]
    position: [0.5998664974825264, 0.33407948970083634, -2.03748629510374, 1.4991680751013094, -1.1649061168362789, -1.9787481268888447, -0.5664763070321123]
    velocity: []
    effort: []
result_type: [2]


In [27]:
target_pose = dict(zip(resp.joints[0].name, resp.joints[0].position))

In [28]:
limb.set_joint_positions(target_pose)

In [29]:
# Euler Angle to Quaternion Example

In [32]:
import tf

In [33]:
zero_angle_quaternion= tf.transformations.quaternion_from_euler(0,0,0)

In [34]:
print zero_angle_quaternion

[0. 0. 0. 1.]
